In [1]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt  
import numpy as np 


In [2]:
df = pd.read_csv("gurgaon_properties_post_feature_selection_v2 (1).csv")

In [3]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3.0,2.0,2,New Property,850.0,0.0,0.0,0.0,Low,Low Floor
1,flat,sector 89,0.95,2.0,2.0,2,New Property,1226.0,1.0,0.0,0.0,Low,Mid Floor
2,flat,sohna road,0.32,2.0,2.0,1,New Property,1000.0,0.0,0.0,0.0,Low,High Floor
3,flat,sector 92,1.60,3.0,4.0,3+,Relatively New,1615.0,1.0,0.0,1.0,High,Mid Floor
4,flat,sector 102,0.48,2.0,2.0,1,Relatively New,582.0,0.0,1.0,0.0,High,Mid Floor


In [4]:
df.drop(columns=['store room', 'balcony', 'floor_category'], inplace=True)

In [5]:
df.head()

,property_type,sector,price,bedRoom,bathroom,agePossession,built_up_area,servant room,furnishing_type,luxury_category
0,flat,sector 36,0.82,3.0,2.0,New Property,850.0,0.0,0.0,Low
1,flat,sector 89,0.95,2.0,2.0,New Property,1226.0,1.0,0.0,Low
2,flat,sohna road,0.32,2.0,2.0,New Property,1000.0,0.0,0.0,Low
3,flat,sector 92,1.60,3.0,4.0,Relatively New,1615.0,1.0,1.0,High
4,flat,sector 102,0.48,2.0,2.0,Relatively New,582.0,0.0,0.0,High


In [14]:
cat_col = df.select_dtypes(include = ['object']).columns.tolist()
print(cat_col)

['property_type', 'sector', 'agePossession', 'luxury_category']


In [15]:
# we have to applly transformations on categorical columns 

In [18]:
# num = ['price', 'bedroom', 'bathroom', 'builtup area', 'servant room']
# ordinal = ["property_type", "furnishing_type", "luxury_category"] 
# ohe = ["sector", "agepossesion"]

In [17]:
df['furnishing_type'].value_counts()

furnishing_type
0.0    2349
1.0    1018
2.0     187
Name: count, dtype: int64

In [25]:
df['agePossession'].value_counts()

agePossession
New Property          2331
Old Property           946
Under Construction     277
Name: count, dtype: int64

In [21]:
df['agePossession'] = df['agePossession'].replace({'Relatively New': 'New Property', "Moderately Old": "Old Property"})

In [22]:
df['property_type'].value_counts()

property_type
flat     2804
house     750
Name: count, dtype: int64

In [24]:
df['property_type'] = df['property_type'].replace({'flat' : 0, 'house' : 1})

C:\Users\dhruv\AppData\Local\Temp\ipykernel_18408\2997991399.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['property_type'] = df['property_type'].replace({'flat' : 0, 'house' : 1})


In [26]:
df['luxury_category'].value_counts()

luxury_category
Low       1594
Medium    1465
High       495
Name: count, dtype: int64

In [28]:
df['luxury_category'] = df['luxury_category'].replace({"Low" : 0, "Medium" : 1, "High" : 2})

C:\Users\dhruv\AppData\Local\Temp\ipykernel_18408\3311176827.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['luxury_category'] = df['luxury_category'].replace({"Low" : 0, "Medium" : 1, "High" : 2})


In [30]:
new_df = pd.get_dummies(df, columns = ['sector', 'agePossession'], drop_first=True)

In [31]:
new_df.shape

(3554, 113)

In [33]:
X = new_df.drop(columns='price') 
y = new_df['price']
y_log = np.log1p(y)

In [34]:
y_log

0       0.598837
1       0.667829
2       0.277632
3       0.955511
4       0.392042
          ...   
3549    0.314811
3550    1.945910
3551    0.470004
3552    2.803360
3553    1.022451
Name: price, Length: 3554, dtype: float64

In [36]:
# X mei scaling karna padega 

In [39]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_scaled = sc.fit_transform(X)

In [40]:
df.columns

Index(['property_type', 'sector', 'price', 'bedRoom', 'bathroom',
       'agePossession', 'built_up_area', 'servant room', 'furnishing_type',
       'luxury_category'],
      dtype='object')

In [45]:
X_scaled = pd.DataFrame(data = X_scaled, columns=X.columns)

In [51]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression

In [52]:
kfold = KFold(n_splits=5, shuffle =True, random_state=42)

In [54]:
scores = cross_val_score(LinearRegression(),X_scaled, y_log, scoring='r2', cv = kfold)

In [55]:
scores

array([0.86721493, 0.86453835, 0.84041505, 0.84470579, 0.83808155])

In [56]:
lr = LinearRegression() 
lr.fit(X_scaled, y_log)

LinearRegression()

In [58]:
lr.coef_.shape

(112,)

In [70]:
coef_df = pd.DataFrame(data = lr.coef_.reshape(1,112), columns=X_scaled.columns).stack().reset_index().drop(columns = 'level_0').rename({'0':'coef', 'level_1':'feature'})

In [76]:
coef_df = coef_df.rename(columns={'level_1' :'feature',0:'coef'})

In [77]:
coef_df.sort_values(by = 'coef', ascending=False)

,feature,coef
3,built_up_area,0.210638
0,property_type,0.120165
35,sector_sector 26,0.073883
57,sector_sector 50,0.071822
73,sector_sector 65,0.071046
...,...,...
47,sector_sector 4,-0.020775
8,sector_manesar,-0.022714
107,sector_sector 95,-0.025222
105,sector_sector 92,-0.028556
